## This notebook gives instructions on advanced scenarios

In [ ]:
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from IPython.display import clear_output
import time


In [2]:
from ray_on_aml.core import Ray_On_AML

ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="d15-v2",additional_pip_packages=['torch==1.10.0', 'torchvision', 'sklearn'], maxnode=1)
ray = ray_on_aml.getRay()
# Note that by default, ci_is_head=True which means  compute instance as head node and all nodes in the remote compute cluster as workers 
# But if you want to use one of the nodes in the remote AML compute cluster is used as head node and the remaining are worker nodes.
# then simply specify ray = ray_on_aml.getRay(ci_is_head=False)
# To install additional library, use additional_pip_packages and additional_conda_packages parameters.
time.sleep(50)
ray.cluster_resources()

Cancel active AML runs if any
Canceling active run  ray_on_aml_1642801578_7ee37e36 in ray_on_aml
Shutting down ray if any
Found existing cluster d15-v2
Creating new Environment ray-0.0.7-3074610585624962555


2022-01-21 23:08:28,366	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


Waiting for cluster to start


{'CPU': 26.0,
 'object_store_memory': 60333033062.0,
 'memory': 135397421466.0,
 'GPU': 1.0,
 'node:10.0.0.13': 1.0,
 'accelerator_type:K80': 1.0,
 'node:10.0.0.17': 1.0}

## Loading custom resources such as files and python modules to run interactively

In [3]:
#Create a new client object
import ray
from files import utils 
client = ray.init(f"ray://{ray_on_aml.get_ip()}:10001", runtime_env={"py_modules": ['files']})


@ray.remote
class test():
    def test_method(self):
        return utils.Foo().bar()
        return(os.listdir())
test_obj = test.remote()
print(ray.get(test_obj.test_method.remote()))
client.disconnect() 
#disconnect after use, if you don't do this, you will have error if you run the client connection again


2022-01-21 23:09:46,587	INFO packaging.py:352 -- Creating a file package for local directory 'files'.
2022-01-21 23:09:46,810	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_fdd168c6e168930f.zip' (0.00MiB) to Ray cluster...
2022-01-21 23:09:46,812	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_fdd168c6e168930f.zip'.
The autoscaler failed with the following error:
Terminated with signal 15
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 543, in <module>
    monitor.run()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 439, in run
    self._run()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 346, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



(1, 2)


Learn more at https://docs.ray.io/en/latest/handling-dependencies.html